# Importing the IntAct database into postgreSQL

This program imports data from the REST interface of the Intact database. By running this program the following
steps are performed :

- make sure the database and all its tables exist. If not, the program will create them.
- Truncate/empty the existing database.
- Import all data for taxid:83333 (as specified in the url parameter)

Alternatively it is also possible to import data from a text file. This is typically useful for testing purposes.
For this the call to import_from_url should be commented out and the comment before import_from_file should be removed.
Also make sure the path variable is set to the correct path.

Format specification : http://psicquic.github.io/MITAB27Format.html

Rijksuniversiteit Groningen, 2018

C.M. Punter (c.m.punter@rug.nl)

## Imports, connecting to postgreSQL and assigning variables

In [5]:
# importing modules

import psycopg2
import urllib.request
import os

In [6]:
# specify the username, password and the name of the database to be created/modified

username = 'postgres'
password = 'password'
database = 'intact_20201208'

# only one of the url/path acess can be used, comment out unwanted line

# specify path to the database file

path = os.path.join(os.getcwd(), 'intact_20201208.txt')

# specify the database url

# url = 'http://www.ebi.ac.uk/Tools/webservices/psicquic/intact/webservices/current/search/query/taxid:83333?format=tab27'

conn = None

## Functions

In [7]:
# connect to postgreSQL

def connect():
    global conn
    conn = psycopg2.connect("dbname=%s user=%s password=%s" % ('postgres', username, password))
    conn.autocommit = True
    cur = conn.cursor()
    cur.execute("select 1 from pg_database where datname = %s", (database,))

    if not cur.fetchone():
        cur.execute("create database %s" % database)
        conn.commit()

    conn.close()
    conn = psycopg2.connect("dbname=%s user=%s password=%s" % (database, username, password))

# create SQL tables out of the imported file
    
def create_tables():
    cur = conn.cursor()

    cur.execute("select 1 from pg_type where typname = 'identifier_type'")

    if not cur.fetchone():
        cur.execute("create type identifier_type as enum(" +
                    "'interactor', " +
                    "'alt_interactor', " +
                    "'alias_interactor', " +
                    "'detection_method'," +
                    "'author', " +
                    "'publication', " +
                    "'taxonomy', " +
                    "'interaction_type', " +
                    "'source_database', " +
                    "'interaction', " +
                    "'confidence_score', " +
                    "'complex_expansion', " +
                    "'biological_role', " +
                    "'experimental_role', " +
                    "'interactor_type', " +
                    "'xref_interaction', " +
                    "'xref_interactor', " +
                    "'annotations_interactor', " +
                    "'annotations_interaction', " +
                    "'ncbi_taxonomy', " +
                    "'parameters_interaction', " +
                    "'creation_date', " +
                    "'update_date', " +
                    "'checksum_interactor', " +
                    "'checksum_interaction', " +
                    "'negative', " +
                    "'feature_interactor', " +
                    "'stoichiometry_interactor', " +
                    "'participant_method_interactor')")

    cur.execute('create table if not exists databases (' +
                'id serial primary key, ' +
                'name text unique)')

    cur.execute('create table if not exists identifiers (' +
                'id serial primary key, ' +
                'type identifier_type, ' +
                'database_id integer references databases on delete cascade, ' +
                'identifier text, ' +
                'unique(type, database_id, identifier))')

    cur.execute('create table if not exists interactions (' +
                'id serial primary key, ' +
                'confidence_score real)')

    cur.execute('create table if not exists interaction_identifier ('
                'interaction_id integer references interactions on delete cascade, ' +
                'identifier_id integer references identifiers on delete cascade, '
                'ab char,' +
                'unique(interaction_id, identifier_id, ab))')

    conn.commit()

# delet given database
    
def truncate_tables():
    cur = conn.cursor()
    cur.execute('truncate databases, identifiers, interactions, interaction_identifier cascade')
    conn.commit()

# get database ID
    
def get_database_id(database_name):
    cur = conn.cursor()
    cur.execute('select id from databases where name = %s', (database_name,))
    return cur.fetchone()

# import database line
# lines contain varied information that can, in itself, be split into columns



def import_line(line):
    columns = line.split('\t')

    if len(columns) != 42:
        print('number of columns is %d and not 42, skipping row:', len(columns))
        print(line)
        return False

    confidence_score = None;

    if columns[14] != '-':
        confidence_score = float(columns[14].rsplit(':', 1)[1])

    cur = conn.cursor()
    cur.execute('insert into interactions (confidence_score) values (%s) returning id', (confidence_score,))
    interaction_id = cur.fetchone()

    column_types = [('interactor', 'a'),                        # 1
                    ('interactor', 'b'),                        # 2
                    ('alt_interactor', 'a'),                    # 3
                    ('alt_interactor', 'b'),                    # 4
                    ('alias_interactor', 'a'),                  # 5
                    ('alias_interactor', 'b'),                  # 6
                    ('detection_method', None),                 # 7
                    ('author', None),                           # 8
                    ('publication', None),                      # 9
                    ('taxonomy', 'a'),                          # 10
                    ('taxonomy', 'b'),                          # 11
                    ('interaction_type', None),                 # 12
                    ('source_database', None),                  # 13
                    ('interaction', None),                      # 14
                    ('confidence_score', None),                 # 15
                    ('complex_expansion', None),                # 16
                    ('biological_role', 'a'),                   # 17
                    ('biological_role', 'b'),                   # 18
                    ('experimental_role', 'a'),                 # 19
                    ('experimental_role', 'b'),                 # 20
                    ('interactor_type', 'a'),                   # 21
                    ('interactor_type', 'b'),                   # 22
                    ('xref_interactor', 'a'),                   # 23
                    ('xref_interactor', 'b'),                   # 24
                    ('xref_interaction', None),                 # 25
                    ('annotations_interactor', 'a'),            # 26
                    ('annotations_interactor', 'b'),            # 27
                    ('annotations_interaction', None),          # 28
                    ('ncbi_taxonomy', None),                    # 29
                    ('parameters_interaction', None),           # 30
                    ('creation_date', None),                    # 31
                    ('update_date', None),                      # 32
                    ('checksum_interactor', 'a'),               # 33
                    ('checksum_interactor', 'b'),               # 34
                    ('checksum_interaction', None),             # 35
                    ('negative', None),                         # 36
                    ('feature_interactor', 'a'),                # 37
                    ('feature_interactor', 'b'),                # 38
                    ('stoichiometry_interactor', 'a'),          # 39
                    ('stoichiometry_interactor', 'b'),          # 40
                    ('participant_method_interactor', 'a'),     # 41
                    ('participant_method_interactor', 'b'),     # 42
                    ]

    for (type, ab), column in zip(column_types, columns):

        values = column.split('|')

        for value in values:
            database_name = None
            database_id = None
            identifier = value

            if ':' in value:
                database_name, identifier = value.split(':', 1)

            if database_name:
                database_id = get_database_id(database_name)

                if not database_id:
                    cur.execute('insert into databases (name) values (%s) returning id', (database_name,))
                    database_id = cur.fetchone()

            cur.execute('select id from identifiers where type = %s and database_id = %s and identifier = %s', (type, database_id, identifier))
            identifier_id = cur.fetchone()

            if not identifier_id:
                cur.execute('insert into identifiers (type, database_id, identifier) values (%s, %s, %s) returning id', (type, database_id, identifier))
                identifier_id = cur.fetchone()

            cur.execute('insert into interaction_identifier (interaction_id, identifier_id, ab) values (%s, %s, %s) on conflict do nothing', (interaction_id, identifier_id, ab))

    conn.commit()

    return True

# importing database from a file

def import_from_file(path, skip_header=False):
    import_count = 0
    skip_count = 0

    with open(path, encoding="utf8") as f:

        if skip_header:
            next(f)

        for line in f:

            line = line.rstrip()  # remove \n at the end of the line
            if import_line(line):
                import_count += 1
            else:
                skip_count += 1

    print('imported %d rows (interactions) and skipped %d rows (interactions), total rows (interactions) is %d' % (
    import_count, skip_count, import_count + skip_count))

# importing database from url
    
def import_from_url(url):
    import_count = 0
    skip_count = 0

    with urllib.request.urlopen(url) as f:
        for line in f:
            line = line.decode('utf-8')
            line = line.rstrip()  # remove \n at the end of the line
            if import_line(line):
                import_count += 1
            else:
                skip_count += 1

    print('imported %d rows (interactions) and skipped %d rows (interactions), total rows (interactions) is %d' % (
    import_count, skip_count, import_count + skip_count))

## Execution

In [8]:
connect()           # connect to database
create_tables()     # make sure all tables exist
truncate_tables()   # truncate tables (remove all rows from all tables)


# only one of the url/path acess can be used, comment out unwanted line

import_from_file(path, True)

# import_from_url(url)

print('done!')

imported 29417 rows (interactions) and skipped 0 rows (interactions), total rows (interactions) is 29417
done!
